In [1]:
from ultralytics import YOLO
import os
import shutil

# Paths
custom_export_dir = '/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/quantization'
os.makedirs(custom_export_dir, exist_ok=True)

# Load trained model
model = YOLO('/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/best.pt')

# Export to ONNX
onnx_path = model.export(format='onnx', dynamic=True, simplify=True)

# Destination path in custom directory
destination_path = os.path.join(custom_export_dir, 'best.onnx')

# Ensure destination directory exists
os.makedirs(custom_export_dir, exist_ok=True)

# Move the ONNX model to the custom directory
shutil.move(onnx_path, destination_path)  # Moving the file to avoid cross-filesystem issues

print(f"Model saved to: {destination_path}")


Ultralytics 8.3.103 🚀 Python-3.12.10 torch-2.6.0+cu124 CPU (Intel Xeon Gold 6226R 2.90GHz)
YOLO11n-seg summary (fused): 113 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs

PyTorch: starting from '/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (5.7 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.51...
ONNX: export success ✅ 16.7s, saved as '/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/best.onnx' (11.0 MB)

Export complete (17.5s)
Results saved to /student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights
Predict:         yolo predict task=segment model=/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-s

In [7]:
pip install onnx onnxruntime onnxruntime-tools

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from onnxruntime.quantization import quantize_dynamic, QuantType
import os

# Define target directory
output_dir = '/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/quantization/'
os.makedirs(output_dir, exist_ok=True)

# Set paths
input_model = '/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/quantization/best.onnx'
output_model = os.path.join(output_dir, 'best-quant.onnx')

# Perform quantization
quantize_dynamic(
    model_input=input_model,
    model_output=output_model,
    weight_type=QuantType.QUInt8  # or QuantType.QUInt8
)

print(f"Quantization complete! The quantized model is saved to: {output_model}")

Quantization complete! The quantized model is saved to: /student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/quantization/best-quant.onnx


In [ ]:
import os
import time
import onnxruntime as ort
from PIL import Image
import numpy as np
import cv2

# === CONFIGURATION ===
onnx_model_path = '/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/quantization/best-quant.onnx'  # Path to the ONNX model
image_path = '/student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/images/test/video01_video01_28900_frame_28914_endo.png'  # Path to the test image

# === IMAGE PREPROCESSING ===
def preprocess_image(image_path, img_size=(640, 640)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(img_size)
    img = np.array(image).astype(np.float32) / 255.0  # Normalize to [0, 1]
    img = img.transpose(2, 0, 1)  # Convert to CHW format (Channels, Height, Width)
    return np.expand_dims(img, axis=0)  # Add batch dimension

# === NON-MAXIMUM SUPPRESSION ===
def non_maximum_suppression(boxes, confidences, threshold=0.5):
    boxes = boxes.reshape(-1, 4)
    confidences = confidences.flatten()
    indices = cv2.dnn.NMSBoxes(boxes.tolist(), confidences.tolist(), score_threshold=threshold, nms_threshold=0.4)
    return indices.flatten() if indices is not None and len(indices) > 0 else []

# === INFERENCE FUNCTION FOR ONNX MODEL ===
def run_onnx_model(onnx_model_path, image_path):
    session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])
    img = preprocess_image(image_path)  # Preprocess the input image
    input_name = session.get_inputs()[0].name
    start_time = time.time()
    outputs = session.run(None, {input_name: img})
    inference_time = time.time() - start_time
    return outputs, inference_time

# === VISUALIZE INFERENCE RESULTS ===
def visualize_inference(image_path, boxes, confidences, labels):
    image = cv2.imread(image_path)
    for i in range(len(boxes)):
        box = boxes[i]
        x1, y1, x2, y2 = map(int, box)  # Ensure box has 4 values
        confidence = float(confidences[i])
        label = labels[i] if labels is not None else 0  # Use label 0 as fallback
        color = (0, 255, 0)  # Green color for bounding boxes

        # Draw rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        # Add label and confidence score
        cv2.putText(image, f"Class {label} ({confidence:.2f})", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show the image with bounding boxes
    cv2.imshow("Inference Result", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# === RUN INFERENCE ON ONNX MODEL ===
def test_onnx_model(onnx_model_path, image_path):
    print(f"Testing ONNX model: {onnx_model_path}")

    if not os.path.exists(onnx_model_path):
        print(f"Error: Model file not found at {onnx_model_path}")
        return

    # Run inference on the image
    outputs, inference_time = run_onnx_model(onnx_model_path, image_path)

    print(f"Inference Time: {inference_time:.4f} seconds")

    # Extract results for visualization (assuming result contains boxes, confidences, and labels)
    boxes = outputs[0][:, :4].reshape(-1, 4)  # Flatten the boxes
    confidences = outputs[0][:, 4].flatten()  # Flatten the confidence scores
    labels = outputs[0][:, 5].flatten()  # Flatten the labels

    # Ensure that boxes are properly shaped (4 values per box)
    print(f"Boxes Shape: {boxes.shape}, Confidences Shape: {confidences.shape}, Labels Shape: {labels.shape}")

    # Visualize the results
    visualize_inference(image_path, boxes, confidences, labels)


# === RUN THE TEST ===
test_onnx_model(onnx_model_path, image_path)


Testing ONNX model: /student/vcheruku/Enhance-Data-Diversity-and-Robustness/yolo-for-single-class/YOLO11/yolo-on-single-object-detection/weights/quantization/best-quant.onnx
Inference Time: 0.1088 seconds
Boxes Shape: (8400, 4), Confidences Shape: (8400,), Labels Shape: (8400,)
